<a href="https://colab.research.google.com/github/RoanvanMerwijk/4CBLW00-20/blob/main/XGBoost_group3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model and Data Setup

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.model_selection import KFold


In [ ]:
data_frame = pd.read_csv("ward_level_burglary_data_long.csv", on_bad_lines='skip')
data_frame[data_frame['Year'] == 2020]

<ipython-input-2-362407479>:1: DtypeWarning: Columns (8,13) have mixed types. Specify dtype option on import or set low_memory=False.
  data_frame = pd.read_csv("ward_level_burglary_data_long.csv", on_bad_lines='skip')


,WardName,Borough,WardCode,Year,Month,BurglaryCount,Latitude,Longitude,Avg burglaries per month,Population Density,...,edu_london_decile,Income_london_rank_norm,employment_london_rank_norm,edu_london_rank_norm,health_london_rank_norm,barriers_london_rank_norm,livingEnv_london_rank_norm,imd_score_no_crime,imd_no_crime_rank,NearestStopDistance
2503,Homerton,E09000012,E05009376,2020.0,7.0,13.0,51.547361,-0.048489,1.149682,17809.716667,...,4.166667,0.854277,0.771612,0.504291,0.742466,0.887946,0.742349,0.756906,607.500000,125.367209
2504,Hook & Chessington North,E09000021,E05013935,2020.0,7.0,1.0,51.366691,-0.303534,0.372611,5373.266667,...,3.500000,0.436648,0.404651,0.555970,0.266817,0.226279,0.455798,0.401539,3121.666667,114.983073
2505,Hornsey,E09000014,E05013593,2020.0,7.0,9.0,51.587402,-0.121593,0.731905,12292.500000,...,8.111111,0.558944,0.480195,0.162824,0.413790,0.409829,0.719227,0.459888,2671.111111,199.547542
2506,Hounslow Central,E09000018,E05013620,2020.0,7.0,1.0,51.470038,-0.363114,0.808917,13565.150000,...,6.500000,0.605645,0.453548,0.334325,0.482539,0.710333,0.620661,0.521403,2322.750000,100.203092
2507,Hounslow East,E09000018,E05013621,2020.0,7.0,5.0,51.471401,-0.354745,0.876433,11020.360000,...,6.333333,0.550405,0.425439,0.301620,0.452605,0.657840,0.749053,0.499169,2475.333333,79.139083
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118099,Worcester Park North,E09000029,E05013772,2020.0,12.0,4.0,51.381661,-0.233673,0.503185,6525.880000,...,5.600000,0.346704,0.248838,0.373771,0.168356,0.188498,0.392683,0.288337,3795.400000,278.085192
118100,Worcester Park South,E09000029,E05013773,2020.0,12.0,2.0,51.369706,-0.234868,0.571975,5478.680000,...,7.400000,0.168364,0.138029,0.236443,0.135440,0.035002,0.345323,0.170547,4420.600000,203.902297
118101,Wormholt,E09000013,E05013753,2020.0,12.0,1.0,51.510125,-0.238773,1.006369,14293.280000,...,5.800000,0.792517,0.735969,0.375384,0.631562,0.423252,0.798420,0.655024,1339.600000,120.073141
118102,Yeading,E09000017,E05013583,2020.0,12.0,5.0,51.527272,-0.394468,0.970541,6327.187500,...,2.750000,0.701173,0.613053,0.640276,0.463098,0.687655,0.391539,0.601558,1709.375000,172.840919


In [ ]:
target_column = 'BurglaryCount'

In [ ]:
y = data_frame[[target_column]]


X = data_frame.drop(columns=[
    target_column,
'Borough', 'WardName', 'WardCode', "imd_score_no_crime", "imd_no_crime_rank",
"Income_london_decile", "employment_london_decile",	"barriers_london_decile",	"livingEnv_london_decile",	"health_london_decile",	"edu_london_decile",	"Income_london_rank_norm",	"employment_london_rank_norm",	"edu_london_rank_norm",	"health_london_rank_norm",	"barriers_london_rank_norm",	"livingEnv_london_rank_norm",
'Latitude', 'Longitude', 'Avg burglaries per month'
], errors='ignore')
y

,BurglaryCount
0,21.0
1,11.0
2,6.0
3,3.0
4,10.0
...,...
146197,0.0
146198,0.0
146199,4.0
146200,4.0


In [ ]:
X = X.select_dtypes(include=[np.number])
X = X.fillna(0)
X

,Year,Month,Population Density,Income_london_rank,employment_london_rank,barriers_london_rank,livingEnv_london_rank,health_london_rank,edu_london_rank,NearestStopDistance
0,2019.0,3.0,9971.012500,21099.833333,23581.833333,3737.333333,7463.333333,21766.000000,25680.666667,102.251039
1,2019.0,3.0,14302.471429,20670.857143,25090.000000,4153.142857,5304.857143,28938.571429,25411.857143,84.885852
2,2019.0,3.0,8597.255556,8297.444444,10677.666667,2334.333333,12446.555556,12985.111111,13878.000000,124.402045
3,2019.0,3.0,14864.240000,28307.200000,30342.600000,1854.000000,2948.600000,32130.000000,31843.800000,119.368572
4,2019.0,3.0,7776.728571,18162.428571,19596.000000,3228.285714,11422.000000,19866.571429,21789.571429,106.215074
...,...,...,...,...,...,...,...,...,...,...
146197,2025.0,4.0,6525.880000,21515.000000,24745.800000,3923.800000,17530.600000,27521.600000,21258.600000,278.085192
146198,2025.0,4.0,5478.680000,27339.400000,28350.200000,4665.800000,18897.600000,28562.200000,25515.200000,203.902297
146199,2025.0,4.0,14293.280000,6955.200000,8900.400000,2789.000000,5819.400000,12877.800000,21208.600000,120.073141
146200,2025.0,4.0,6327.187500,9938.375000,12898.625000,1510.875000,17563.625000,18203.625000,12998.000000,172.840919


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)
y_test

,BurglaryCount
33947,2.0
110263,4.0
87674,11.0
74136,5.0
140082,3.0
...,...
126333,12.0
118329,7.0
89747,17.0
64970,6.0


In [ ]:
model = xgb.XGBRegressor(
    subsample = 0.55,
    reg_alpha = 0.15,
    reg_lambda = 20,
    n_estimators = 400,
    max_depth = 12,
    learning_rate = 0.02,
    gamma = 5,
    colsample_bytree = 0.9
    )
model.fit(X_train, y_train)

XGBoostError: [10:16:27] /workspace/src/data/data.cc:514: Check failed: valid: Label contains NaN, infinity or a value too large.
Stack trace:
  [bt] (0) /usr/local/lib/python3.11/dist-packages/xgboost/lib/libxgboost.so(+0x25c1ac) [0x7d5f6d45c1ac]
  [bt] (1) /usr/local/lib/python3.11/dist-packages/xgboost/lib/libxgboost.so(+0x4e2229) [0x7d5f6d6e2229]
  [bt] (2) /usr/local/lib/python3.11/dist-packages/xgboost/lib/libxgboost.so(+0x4e3a60) [0x7d5f6d6e3a60]
  [bt] (3) /usr/local/lib/python3.11/dist-packages/xgboost/lib/libxgboost.so(XGDMatrixSetInfoFromInterface+0xb2) [0x7d5f6d363522]
  [bt] (4) /lib/x86_64-linux-gnu/libffi.so.8(+0x7e2e) [0x7d5fd9419e2e]
  [bt] (5) /lib/x86_64-linux-gnu/libffi.so.8(+0x4493) [0x7d5fd9416493]
  [bt] (6) /usr/lib/python3.11/lib-dynload/_ctypes.cpython-311-x86_64-linux-gnu.so(+0xa4d8) [0x7d5fd967a4d8]
  [bt] (7) /usr/lib/python3.11/lib-dynload/_ctypes.cpython-311-x86_64-linux-gnu.so(+0x9c8e) [0x7d5fd9679c8e]
  [bt] (8) /usr/bin/python3(_PyObject_MakeTpCall+0x2

# Evaluation

In [ ]:
y_pred = model.predict(X_test)
print("MAE:", mean_absolute_error(y_test, y_pred))
print("MSE:", mean_squared_error(y_test, y_pred))
print("R² Score:", r2_score(y_test, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))

In [ ]:
xgb.plot_importance(model, max_num_features=10)
plt.title("Top 10 Most Important Features")
plt.show()

In [ ]:
xgb_reg = xgb.XGBRegressor(
    objective='reg:squarederror',
    random_state=42,
    verbosity=0
)

param_dist = {
    'n_estimators': [50, 100, 400, 60],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7, 10, 8, 15],
    'subsample': [0.5, 0.7, 0.9, 1.0],
    'colsample_bytree': [0.5, 0.9, 1.0],
    'gamma': [0, 0.5, 1, 5],
    'reg_alpha': [0, 0.1, 0.5, 1.0],
    'reg_lambda': [1, 1.5, 2, 5]
}

kf = KFold(n_splits=4, shuffle=True, random_state=42)

random_search = RandomizedSearchCV(
    estimator=xgb_reg,
    param_distributions=param_dist,
    n_iter=400,
    scoring='neg_mean_squared_error',
    cv=kf,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

random_search.fit(X_train, y_train)

print("Best parameters found:")
for key, val in random_search.best_params_.items():
    print(f"  • {key}: {val}")

for fold_idx, (train_idx, val_idx) in enumerate(kf.split(X_train)):
    print(f"\nFold {fold_idx}")
    print("  TRAIN indices sample:", train_idx[:10], "... (total", len(train_idx), ")")
    print("  VALID indices sample:", val_idx[:10], "... (total", len(val_idx), ")")

cv_df = pd.DataFrame(random_search.cv_results_)
display(cv_df[[
    'params',
    'split0_test_score',
    'split1_test_score',
    'split2_test_score',
    'mean_test_score'
]].sort_values('mean_test_score', ascending=False).head())

best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test)

print("\nEvaluation on the HOLDOUT set:")
print(f"MAE:  {mean_absolute_error(y_test, y_pred):.4f}")
print(f"MSE:  {mean_squared_error(y_test, y_pred):.4f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred)):.4f}")
print(f"R²:   {r2_score(y_test, y_pred):.4f}")

In [ ]:
model.fit(X_train, y_train)

In [ ]:
train_data, test_data = train_test_split(df, test_size=0.3, random_state=42)

# Predict
y_pred = model.predict(X_test)
y_test = y_test.squeeze()
id_test = X_test.index.to_series().reset_index(drop=True)
residuals = y_test - y_pred
std_dev = np.std(residuals)
lower_bound = y_pred - 1.96 * std_dev
upper_bound = y_pred + 1.96 * std_dev
classification = pd.cut(y_pred, bins=[-np.inf, 5, 15, np.inf], labels=["Low", "Medium", "High"])

# Assemble final table
results_df = id_test.copy()
results_df["Predicted"] = y_pred
results_df["Lower_Bound"] = lower_bound
results_df["Upper_Bound"] = upper_bound
results_df["Classification"] = classification
#results_df["ES_forecast"] = test_data["ES_forecast"].reset_index(drop=True)

print("\n Evaluation on April 2025 with ES_forecast:")
print(f"MAE:  {mean_absolute_error(y_test, y_pred):.4f}")
print(f"MSE:  {mean_squared_error(y_test, y_pred):.4f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred)):.4f}")
print(f"R²:   {r2_score(y_test, y_pred):.4f}")

In [ ]:
# === Set the month/year to predict ===
PRED_YEAR = 2019
PRED_MONTH = 3

# === Reload data to avoid leakage ===
data = pd.read_csv("ward_level_burglary_data_long.csv", on_bad_lines='skip')

# === Filter data for the target month ===
month_data = data[(data["Year"] == PRED_YEAR) & (data["Month"] == PRED_MONTH)].copy()

# === Store identifying columns ===
id_cols = month_data[["WardCode", "WardName", "Borough", "Year", "Month"]].copy()

# === Prepare features (must match training features) ===
# Select only the columns that were used for training
X_pred = month_data[X_train.columns].copy()

# Ensure the order of columns is the same as in X_train
X_pred = X_pred[X_train.columns]

# Handle any missing values by filling with 0, consistent with previous steps
X_pred = X_pred.select_dtypes(include=[np.number]).fillna(0)


# === Make predictions ===
y_pred = model.predict(X_pred)

# === Compute prediction intervals based on residual std dev ===
# Use training residuals from your latest model (computed previously)
# If not available, retrain the model and get residuals
# Assuming 'model' is the best model trained in the previous steps
y_train_pred = model.predict(X_train)
residual_std = np.std(y_train.squeeze() - y_train_pred)


lower_bound = y_pred - 1.96 * residual_std
upper_bound = y_pred + 1.96 * residual_std

# Set lower_bound values less than 0 to 0
lower_bound[lower_bound < 0] = 0

# === Classification based on predicted value ===
classification = pd.cut(
    y_pred,
    bins=[-np.inf, 5, 15, np.inf],
    labels=["Low", "Medium", "High"]
)

# === Final results table ===
id_cols["Predicted_Burglaries"] = np.round(y_pred).astype(int)
id_cols["Lower_Bound"] = lower_bound
id_cols["Upper_Bound"] = upper_bound
id_cols["Classification"] = classification

# === Export to CSV ===
output_filename = f"predicted_burglaries_{PRED_YEAR}_{PRED_MONTH}.csv"
id_cols.to_csv(output_filename, index=False)
print(f" Predictions saved to: {output_filename}")